# Buildathon Summary

* Data
    * Neuroprobe
    * Splits
* Processing
    * Buffer Sampler
    * Re-referencing Transform
    * Patching Transform
    * Masking
* Training
    * Pipeline and Baseline models

## Data

### Neuroprobe

We added Neuroprobe to brainsets!!!

Get the data with:
`brainsets prepare neuroprobe_2025`

In [ ]:
from temporaldata import Data
import h5py

path = "Path to the data"

# Load the data from the h5py file
with h5py.open(path) as f:
    data = Data.from_hdf5(lazy=True)

: 

### Splits

There are a lot of tasks in the dataset, with each task having two splits of two folds.
The existing splitting functionality wasn't enough so we added a better alternative.

In [ ]:
# Get your splits for task_1, fold_1
data.splits.task_1.fold_1

## Processing

### Buffered Sampler

Allows you to do preprocessing/filtering on the fly by avoiding edge effects for common filters.This means less storage use to test multiple processing parameters simulataneously.

Planning to implement: 
* Multitaper Spectrograms
* Fourier Transforms
* Hipass, low-pass, notch...
* Downsampling

In [ ]:
from torch_brain.data.buffer_sampler import BufferedSampler
from torch_brain.data.sampler import RandomFixedWindowSampler


fake_sampling_intervals = make_fake_sampling_intervals(num_records=1)
for rec_id, domain in fake_sampling_intervals.items():
    print(rec_id, domain.start, domain.end)

sampler = RandomFixedWindowSampler(
    sampling_intervals=fake_sampling_intervals,
    window_length=1.0,
)

buffered_sampler = BufferedSampler(
    base_sampler = sampler,
    sampling_intervals=fake_sampling_intervals,
    buffer_len =1.0,
)

sample = next(iter(buffered_sampler))

### Re-referencing
 Bipolar, Laplacian, Common Average Reference, etc.

In [ ]:
from torch_brain.transforms.rereferencing import Rereferencing

reref_transform = Rereferencing(type="laplace")

sample = reref_transform(sample)

: 

### Patching

Multiple people implemented this on their side so it is important to have it as a part of the package.

In [ ]:
from torch_brain.transforms.patching import RegularPatching

patching_transform = RegularPatching(patch_duration=1.0, stride=0.5)

sample = patching_transform(sample)
print(sample.neural_data.shape)

### Masking

Implements three different types of masks: 
* Random Time Masking
* Random Channel Masking
* Random Block Masking

In [ ]:
from torch_brain.transforms.masking import TimeMasking, ChannelMasking, BlockMasking

mask_percentage = 0.35

time_masking_transform = TimeMasking(mask_percentage=mask_percentage, window_duration=0.1)
channel_masking_transform = ChannelMasking(mask_percentage=mask_percentage)
block_masking_transform = BlockMasking(mask_percentage=mask_percentage, time_block_size=0.1, channel_block_size=4)

sample = time_masking_transform(sample)
# sample = channel_masking_transform(sample)
# sample = block_masking_transform(sample)

print(sample.neural_data.mask.shape)

## Training Pipeline and Baseline Models

Training pipeline in `torch_brain/examples` and baseline models that consist of:
* MLP
* CNN

Pipeline makes it easy to add new models and run neuroprobe benchmark.

In [ ]:
# TODO: Add code snippet here

: 

### Future direction

Support exporting the results directly to the leaderboard